![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [2]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

Inspect & Understand data

In [3]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [4]:
ds_jobs_transformed.describe(include='all')

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
count,19158.000000,19158,19158.000000,14650,19158,18772,18698,16345,19093,13220,13018,18735,19158.000000,19158.000000
unique,NaN,123,NaN,3,2,3,5,6,22,8,6,6,NaN,NaN
top,NaN,city_103,NaN,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1,NaN,NaN
freq,NaN,4355,NaN,13221,13792,13817,11598,14492,3286,3083,9817,8040,NaN,NaN
mean,16875.358179,NaN,0.828848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.366896,0.249348
std,9616.292592,NaN,0.123362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.058462,0.432647
min,1.000000,NaN,0.448000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000
25%,8554.250000,NaN,0.740000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.000000,0.000000
50%,16982.500000,NaN,0.903000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.000000,0.000000
75%,25169.750000,NaN,0.920000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.000000,0.000000


Identifying unique values in each column

In [5]:
for column in ds_jobs.columns:
    print(f"{column}: {ds_jobs[column].unique()[:10]}")

student_id: [ 8949 29725 11561 33241   666 21651 28806   402 27107   699]
city: ['city_103' 'city_40' 'city_21' 'city_115' 'city_162' 'city_176'
 'city_160' 'city_46' 'city_61' 'city_114']
city_development_index: [0.92  0.776 0.624 0.789 0.767 0.764 0.762 0.913 0.926 0.827]
gender: ['Male' nan 'Female' 'Other']
relevant_experience: ['Has relevant experience' 'No relevant experience']
enrolled_university: ['no_enrollment' 'Full time course' nan 'Part time course']
education_level: ['Graduate' 'Masters' 'High School' nan 'Phd' 'Primary School']
major_discipline: ['STEM' 'Business Degree' nan 'Arts' 'Humanities' 'No Major' 'Other']
experience: ['>20' '15' '5' '<1' '11' '13' '7' '17' '2' '16']
company_size: [nan '50-99' '<10' '10000+' '5000-9999' '1000-4999' '10-49' '100-499'
 '500-999']
company_type: [nan 'Pvt Ltd' 'Funded Startup' 'Early Stage Startup' 'Other'
 'Public Sector' 'NGO']
last_new_job: ['1' '>4' 'never' '4' '3' '2' nan]
training_hours: [ 36  47  83  52   8  24  18  46 123  32

### Transformation Plan

##### Binary Categorical to Boolean (bool)
- relevant_experience: Transformed to bool indicating whether the student has relevant experience.
- job_change: Transformed to bool indicating whether the student has changed jobs.

##### Integers to 32-bit integers (int32)
- student_id: Transformed to int32.
- training_hours: Transformed to int32.
- 
##### Floats to 16-bit floats (float16)
- city_development_index: Transformed to float16.

##### Nominal Categorical to Category (category)
- city
- gender
- major_discipline
- company_type

##### Ordered Categorical to Ordered Category
- enrolled_university: Transformed to an ordered category with the order ['no_enrollment', 'Part time course', 'Full time course'].
- education_level: Transformed to an ordered category with the order ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'].
- experience: Transformed to an ordered category with the order ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'].
- company_size: Transformed to an ordered category with the order ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'].
- last_new_job: Transformed to an ordered category with the order ['never', '1', '2', '3', '4', '>4'].

1 - Convert Col with 2 Factors to Boolean

In [6]:
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].map({'Has relevant experience': True, 'No relevant experience': False})

In [7]:
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype('bool')

In [8]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  bool   
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  bool   
dtypes: bool(2), float64(1), int64(2), obje

2 - Convert Integer Col to 32-bit integer

In [9]:
ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('int32')

ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('int32')

3 - Convert Float Col to 16-bit Float

In [10]:
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')

4 - Convert Nominal Cat Col to Category

In [11]:
nominal_columns = ['city', 'gender', 'education_level', 'major_discipline', 'company_type']
for col in nominal_columns:
    ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')

5 - Order Col with Ordinal Cat Data

Transform 'enrolled_university' to an ordered category

In [12]:
ds_jobs_transformed['enrolled_university'] = pd.Categorical(
    ds_jobs_transformed['enrolled_university'], 
    categories=['no_enrollment', 'Part time course', 'Full time course'], 
    ordered=True
)

Transform 'education_level' to an ordered category

In [13]:
ds_jobs_transformed['education_level'] = pd.Categorical(
    ds_jobs_transformed['education_level'], 
    categories=['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], 
    ordered=True
)

In [14]:
ordinal_columns = {
    'experience': ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

In [15]:
for col, categories in ordinal_columns.items():
    ds_jobs_transformed[col] = pd.Categorical(ds_jobs_transformed[col], categories=categories, ordered=True)

In [16]:

ds_jobs_transformed.sample(20)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
13137,23079,city_16,0.910156,Male,False,no_enrollment,Masters,STEM,10,NaN,NaN,>4,67,False
18338,24033,city_90,0.698242,Male,False,Part time course,Graduate,STEM,5,50-99,NaN,1,20,False
16361,24173,city_11,0.549805,Male,False,no_enrollment,Graduate,No Major,<1,NaN,NaN,4,158,False
16660,27579,city_97,0.924805,Male,True,Full time course,Graduate,STEM,9,100-499,Public Sector,1,111,False
11725,28376,city_65,0.801758,Female,True,no_enrollment,Masters,STEM,12,NaN,NaN,1,88,False
5048,19713,city_114,0.925781,NaN,True,no_enrollment,Graduate,STEM,>20,500-999,Pvt Ltd,>4,59,False
3698,14458,city_114,0.925781,Male,False,Full time course,Masters,STEM,11,10-49,Public Sector,1,11,False
18090,15206,city_114,0.925781,Male,True,no_enrollment,Masters,STEM,12,10000+,Pvt Ltd,1,5,False
9421,30304,city_64,0.666016,Male,True,no_enrollment,Masters,STEM,>20,100-499,Funded Startup,1,42,False
3703,32412,city_144,0.839844,NaN,False,Full time course,Graduate,STEM,5,50-99,Pvt Ltd,1,33,True


Filter Only Student with 10+ years experiences at Company with 1000+ employees 

In [17]:
ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed['experience'].isin(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'])) &
    (ds_jobs_transformed['company_size'].isin(['1000-4999', '5000-9999', '10000+']))
]

In [18]:
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & 
                                  (ds_jobs_transformed['company_size'] >= '1000-4999')]

In [19]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9), flo

Check Memory Usage before and after

In [20]:
ds_jobs.memory_usage(deep=True).sum()

12391709

In [21]:
ds_jobs_transformed.memory_usage(deep=True).sum()

81348